# Hyperparameter tuning job with custom container

In [1]:
# install google service usage/ artifact registry/ cloud build
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    ! pip install google-cloud-service-usage -q
try:
    import google.cloud.artifactregistry_v1 
except ImportError:
    print('You need to pip install google-cloud-artifact-registry')
    ! pip install google-cloud-artifact-registry -q
try:
    import google.cloud.devtools.cloudbuild
except ImportError:
    print("You need to pip install google-cloud-build")
    !pip install google-cloud-build

## Environment  set up

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'my-project-media-campaign-cost'

In [3]:
REGION = 'australia-southeast1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'media_campaign_cost'
BQ_TABLE = 'mcc_train'

# Resources
BASE_IMAGE = 'us-docker.pkg.dev/deeplearning-platform-release/gcr.io/base-cu113.py310'
DEPLOY_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest'
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/scikit-learn-cpu.0-23:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training


In [4]:
# import packages
from google.cloud import aiplatform
from datetime import datetime
#import pkg_sourses
from IPython.display import Markdown as md
from google.cloud import service_usage_v1
from google.cloud.devtools import cloudbuild_v1
from google.cloud import artifactregistry_v1
from google.cloud import storage
from google.cloud import bigquery
from google.protobuf.struct_pb2 import Value
import json
import numpy as np
import pandas as pd


In [5]:
# creat clients
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project=PROJECT_ID)
gcs = storage.Client(project=PROJECT_ID)
su_client = service_usage_v1.ServiceUsageClient()
ar_client = artifactregistry_v1.ArtifactRegistryClient()
cb_client = cloudbuild_v1.CloudBuildClient()

In [6]:
# parameters
TIMESTAMP =datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
DIR = f"temp/{EXPERIMENT}"

In [7]:
URI

'gs://my-project-media-campaign-cost/01/01'

In [8]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)'
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT 

'683212519680-compute@developer.gserviceaccount.com'

In [9]:
!rm -rf {DIR}
!mkdir -p {DIR}

In [10]:
# Experiment Tracking
FRAMEWORK = 'sklearn'
TASK = 'regression'
MODEL_TYPE = 'xgboost'
EXPERIMENT_NAME = f'experiment-{SERIES}-{EXPERIMENT}-{FRAMEWORK}-{TASK}-{MODEL_TYPE}'
RUN_NAME = f'run-{TIMESTAMP}'

## Get Vertex AI Experiment Tensorboard Instance Name

In [11]:
tb = aiplatform.Tensorboard.list(filter = f'labels.series={SERIES}')
if tb:
    tb = tb[0]
else:
    tb = aiplatform.Tensorboard.create(display_name=SERIES, labels = {'series':f'{SERIES}'})

In [12]:
tb.resource_name

'projects/683212519680/locations/australia-southeast1/tensorboards/2936909906998984704'

## Setup Vertex AI Experiments

In [13]:
aiplatform.init(experiment = EXPERIMENT_NAME, experiment_tensorboard = tb.resource_name)

## Training 

In [97]:
script_path = './trainer/xgbtrain.py'
with open(script_path, 'r') as file:
    data = file.read()
md(f"```python\n\n{data}\n```")

```python

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, TargetEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
import pickle
from google.cloud import bigquery 
from google.cloud import aiplatform
from google.cloud import storage
import hypertune
import argparse
import os
import sys
# import argument to local variables
parser = argparse.ArgumentParser()
parser.add_argument('--project_id',dest ='project_id',type=str)
parser.add_argument('--bq_project',dest ='bq_project', type=str)
parser.add_argument('--bq_dataset',dest ='bq_dataset', type=str)
parser.add_argument('--bq_table', dest ='bq_table',type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--experiment', dest='experiment',type=str)
parser.add_argument('--series', dest='series', type=str)
parser.add_argument('--experiment_name',dest='experiment_name',type=str)
parser.add_argument('--run_name', dest = 'run_name', type = str)
parser.add_argument('--seed', dest = 'seed', type = int)
parser.add_argument('--objective', dest='objective', type = str)
parser.add_argument('--eval_metric', dest='eval_metric', type = str)
parser.add_argument('--tree_method', dest='tree_method', type = str)
parser.add_argument('--grow_policy', dest='grow_policy', type = str)
#hyperparameters 
parser.add_argument('--eta', dest="learning_rate", type=float)
parser.add_argument('--max_depth', dest='max_depth',type=int)
parser.add_argument('--alpha', dest='reg_term', type=float)
args = parser.parse_args()
## creat clients 
bq = bigquery.Client(project = args.project_id)
aiplatform.init(project = args.project_id, location = args.region)
hpt = hypertune.HyperTune()
args.run_name = f"{args.run_name}-{hpt.trial_id}"
# vertex AI Experiment
if args.run_name in [run.name for run in aiplatform.ExperimentRun.list(experiment = args.experiment_name)]:
    expRun = aiplatform.ExperimentRun(run_name = args.run_name, experiment = args.experiment_name)
else:
    expRun = aiplatform.ExperimentRun.create(run_name = args.run_name, experiment = args.experiment_name)
expRun.log_params({'experiment':args.experiment, 'series':args.series, 'project_id': args.project_id})
# get schema from bigquery source
query =  f"SELECT * FROM `{args.bq_project}.{args.bq_dataset}.INFORMATION_SCHEMA.COLUMNS`WHERE TABLE_NAME='{args.bq_table}'"
schema = bq.query(query).to_dataframe()
expRun.log_params({'data_source':f"bq://{args.bq_project}.{args.bq_dataset}.{args.bq_table}"})
# get the data from bigquery
# feature engineering: creat home_children_ratio, drop prepared_food since high-correlation to salad_bar
query = f"SELECT store_sales_in_millions_, unit_sales_in_millions_, total_children-num_children_at_home AS independent_children,gross_weight, avg_cars_at_home_approx__1, recyclable_package, low_fat, units_per_case, store_sqft, coffee_bar+video_store+salad_bar+florist AS store_score, cost FROM `{args.bq_project}.{args.bq_dataset}.{args.bq_table}`"
df = bq.query(query).to_dataframe()
# extract target variable and explainatory variables
y = df['cost']
x = df.drop('cost', axis="columns")
# data preprocessing 
num_attrib = ["independent_children","avg_cars_at_home_approx__1", "store_sqft", "store_score"]
cat_attrib = ["recyclable_package", "low_fat"]
preprocess =  ColumnTransformer([("categorical",OneHotEncoder(), cat_attrib),("numerical", TargetEncoder(), num_attrib)],remainder='passthrough')
x_processed = preprocess.fit_transform(x,y)
# split data into train and test data sets
x_train,x_test,y_train,y_test =  train_test_split(x_processed, y, train_size = 0.8, test_size = 0.2, random_state=50)
# xgboost tree 
xgb_params = {
    'seed': args.seed,
    'objective': args.objective,
    'eval_metric': args.eval_metric,
    'eta': args.learning_rate,
    'max_depth': args.max_depth,
    'alpha': args.reg_term,
    'tree_method': args.tree_method,
    'grow_policy': args.grow_policy
}
xgb_model =  XGBRegressor(**xgb_params)
expRun.log_params({'eta':args.learning_rate, 'max_depth': args.max_depth, 'alpha': args.reg_term})
model_xgb = xgb_model.fit(x_train, y_train)
# test evaluations:
y_pred = model_xgb.predict(x_test)
test_msle = mean_squared_log_error(y_test, y_pred)
expRun.log_metrics({'test_msle': test_msle})
# training evaluations
y_pred_training = model_xgb.predict(x_train)
training_msle = mean_squared_log_error(y_train, y_pred_training)
expRun.log_metrics({'training_msle': training_msle})
# report hypertune info back to Vertex AI Training > Hyperparameter Tuning Job
hpt.report_hyperparameter_tuning_metric( hyperparameter_metric_tag =  'Mean_square_log_error', metric_value = test_msle)
file_name = 'xgb_model.pkl'
# Use predefined environment variable to establish model directionary
model_directory = os.environ['AIP_MODEL_DIR']
storage_path = f'/gcs/{model_directory[5:]}'+file_name
os.makedirs(os.path.dirname(storage_path), exist_ok=True)
# output the model save files directly to GCS destination
with open (storage_path,'wb') as f:
    pickle.dump(model_xgb,f)
expRun.log_params({'model.save': storage_path})
expRun.end_run()

```

## Create a custom container with cloud buid

In [98]:
#store resource in cloud storage
bucket = gcs.lookup_bucket(BUCKET)
if not bucket:
    gcs.bucket(BUCKET).create(location=REGION)
SOURCEPATH = f'{SERIES}/{EXPERIMENT}/training'


In [99]:
# copy training code
blob = storage.Blob(f'{SOURCEPATH}/{EXPERIMENT}_trainer/xgbtrain.py', bucket=gcs.bucket(BUCKET))
blob.upload_from_filename(script_path)

In [100]:
# create requirements.txt file for python
requirements = f""" google-cloud-aiplatform
protobuf
db-dtypes>=1.0.0
google-auth>=2.6.0
google-cloud-bigquery>=3.0.1
cloudml-hypertune
xgboost
"""
blob = storage.Blob(f'{SOURCEPATH}/requirements.txt', bucket=gcs.bucket(BUCKET))
blob.upload_from_string(requirements)

In [101]:
# create the Dockerfile 
dockerfile = f'''
FROM {BASE_IMAGE}
WORKDIR /training
# copy requirements and install them
COPY requirements.txt ./
RUN pip install --no-cache-dir --upgrade pip \
  && pip install --no-cache-dir -r requirements.txt
## Copies the trainer code to the docker image
COPY {EXPERIMENT}_trainer/* ./{EXPERIMENT}_trainer/
## Sets up the entry point to invoke the trainer
ENTRYPOINT ["python", "-m", "{EXPERIMENT}_trainer.xgbtrain"]
'''



In [102]:
blob = storage.Blob(f'{SOURCEPATH}/Dockerfile', bucket=gcs.bucket(BUCKET))
blob.upload_from_string(dockerfile)

In [103]:
## create docker image repository 
docker_repo = None
for repo in ar_client.list_repositories(parent = f"projects/{PROJECT_ID}/locations/{REGION}"):
    if f'{PROJECT_ID}' == repo.name.split('/')[-1]:
        docker_repo = repo
        print(f'Retrieved existing repo:{docker_repo.name}')
if not docker_repo:
    operation = ar_client.create_repository(
    request = artifactregistry_v1.CreateRepositoryRequest(
        parent = f'projects/{PROJECT_ID}/locations/{REGION}',
        repository_id =f'{PROJECT_ID}',
        repository = artifactregistry_v1.Repository(
            description=f'A repository for the {EXPERIMENT} experiment',
            name = f'{PROJECT_ID}',
            format_ = artifactregistry_v1.Repository.Format.DOCKER,
            labels = {'series':SERIES, 'experiment':EXPERIMENT}
    )
    )
    )
    print ("Creating Repository ...")
    docker_repo = operation.result()
    print(f'Complete creating repo: {docker_repo.name}')


Retrieved existing repo:projects/my-project-media-campaign-cost/locations/australia-southeast1/repositories/my-project-media-campaign-cost


In [104]:
docker_repo.name, docker_repo.format_.name

('projects/my-project-media-campaign-cost/locations/australia-southeast1/repositories/my-project-media-campaign-cost',
 'DOCKER')

In [105]:
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{docker_repo.name.split('/')[-1]}"

In [106]:
REPOSITORY

'australia-southeast1-docker.pkg.dev/my-project-media-campaign-cost/my-project-media-campaign-cost'

In [107]:
# setup the build config with empty list of steps - these will be added sequentially
build = cloudbuild_v1.Build(
    steps = []
)
# retrieve the source
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/gsutil',
        'args': ['cp', '-r', f'gs://{PROJECT_ID}/{SOURCEPATH}/*','/workspace']
    }
)
# docker build
build.steps.append(
    {
        'name': 'gcr.io/cloud-builders/docker',
        'args': ['build', '-t', f'{REPOSITORY}/{EXPERIMENT}_trainer','/workspace']
    }    
)

# docker push
build.images = [f"{REPOSITORY}/{EXPERIMENT}_trainer"]



operation = cb_client.create_build(
    project_id = PROJECT_ID,
    build = build
)

response = operation.result()


In [108]:
response.status, response.artifacts

(<Status.SUCCESS: 3>,
 images: "australia-southeast1-docker.pkg.dev/my-project-media-campaign-cost/my-project-media-campaign-cost/01_trainer")

## Setup training job

In [109]:
SEED = '50'
OBJECTIVE = 'reg:squaredlogerror'
EVAL_METRIC = 'rmsle'
TREE_METHOD = 'hist' 
GROWPOLICY = 'lossguide'

In [110]:
CMDARGS = [
    "--project_id=" + PROJECT_ID,
    "--bq_project=" + BQ_PROJECT,
    "--bq_dataset=" + BQ_DATASET,
    "--bq_table=" + BQ_TABLE,
    "--region=" + REGION,
    "--experiment=" + EXPERIMENT,
    "--series=" + SERIES,
    "--experiment_name=" + EXPERIMENT_NAME,
    "--run_name=" + RUN_NAME,
    "--seed=" + SEED,
    "--objective=" + OBJECTIVE,
    "--eval_metric=" + EVAL_METRIC,
    "--tree_method=" + TREE_METHOD,
    "--grow_policy=" + GROWPOLICY,
]

MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count":1,
        "machine_spec":MACHINE_SPEC,
        "container_spec":{
            "image_uri": f"{REPOSITORY}/{EXPERIMENT}_trainer",
            "args":CMDARGS,
        },
    }
]

In [111]:
CMDARGS

['--project_id=my-project-media-campaign-cost',
 '--bq_project=my-project-media-campaign-cost',
 '--bq_dataset=media_campaign_cost',
 '--bq_table=mcc_train',
 '--region=australia-southeast1',
 '--experiment=01',
 '--series=01',
 '--experiment_name=experiment-01-01-sklearn-regression-xgboost',
 '--run_name=run-20231227110602',
 '--seed=50',
 '--objective=reg:squaredlogerror',
 '--eval_metric=rmsle',
 '--tree_method=hist',
 '--grow_policy=lossguide']

In [112]:
customJob = aiplatform.CustomJob(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/models/{TIMESTAMP}",
    staging_bucket = f"{URI}/models/{TIMESTAMP}",
    labels = {'series': f'{SERIES}', 'experiment':f'{EXPERIMENT}','experiment_name':f'{EXPERIMENT_NAME}'}
)

## Setup Hyperparameter Tuning Job

In [113]:
METRIC_SPEC = {
    "Mean_square_log_error": "minimize"
}
PARAMETER_SPEC = {
    "eta": aiplatform.hyperparameter_tuning.DoubleParameterSpec(min=1e-2, max=1, scale='linear'),
    "max_depth": aiplatform.hyperparameter_tuning.DiscreteParameterSpec(values=[5,10,15,20],scale='linear'), 
    "alpha":aiplatform.hyperparameter_tuning.DoubleParameterSpec(min=0, max=2, scale="linear")
}


In [114]:
tuningJob = aiplatform.HyperparameterTuningJob(
    display_name = f'{SERIES}_{EXPERIMENT}_{TIMESTAMP}',
    custom_job = customJob,
    metric_spec = METRIC_SPEC,
    parameter_spec = PARAMETER_SPEC,
    max_trial_count = 18,
    parallel_trial_count =3,
    search_algorithm = None,
    labels = {'series':f'{SERIES}', 'experiment':f'{EXPERIMENT}', 'experiment_name':f'{EXPERIMENT_NAME}'}
)

## Run training job

In [115]:
tuningJob.run(
    service_account = SERVICE_ACCOUNT
)
tuningJob.resource_name, tuningJob.display_name

Creating HyperparameterTuningJob
HyperparameterTuningJob created. Resource name: projects/683212519680/locations/australia-southeast1/hyperparameterTuningJobs/6553592877870481408
To use this HyperparameterTuningJob in another session:
hpt_job = aiplatform.HyperparameterTuningJob.get('projects/683212519680/locations/australia-southeast1/hyperparameterTuningJobs/6553592877870481408')
View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/australia-southeast1/training/6553592877870481408?project=683212519680
HyperparameterTuningJob projects/683212519680/locations/australia-southeast1/hyperparameterTuningJobs/6553592877870481408 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/683212519680/locations/australia-southeast1/hyperparameterTuningJobs/6553592877870481408 current state:
JobState.JOB_STATE_PENDING
HyperparameterTuningJob projects/683212519680/locations/australia-southeast1/hyperparameterTuningJobs/6553592877870481408 current s

('projects/683212519680/locations/australia-southeast1/hyperparameterTuningJobs/6553592877870481408',
 '01_01_20231227110602')

In [116]:
job_link = f"https://console.cloud.google.com/ai/platform/locations/{REGION}/training/{tuningJob.resource_name.split('/')[-1]}?project={PROJECT_ID}"

print(f'Review the Job here:\n{job_link}')

Review the Job here:
https://console.cloud.google.com/ai/platform/locations/australia-southeast1/training/6553592877870481408?project=my-project-media-campaign-cost


## Get the best run

In [117]:
mse  =  [trial.final_measurement.metrics[0].value if trial.state.name=='SUCCEEDED' else 1 for trial in tuningJob.trials]
best = tuningJob.trials[mse.index(min(mse))]

In [118]:
tuningJob.trials

[id: "1"
 state: SUCCEEDED
 parameters {
   parameter_id: "alpha"
   value {
     number_value: 1.0
   }
 }
 parameters {
   parameter_id: "eta"
   value {
     number_value: 0.505
   }
 }
 parameters {
   parameter_id: "max_depth"
   value {
     number_value: 15.0
   }
 }
 final_measurement {
   step_count: 1
   metrics {
     metric_id: "Mean_square_log_error"
     value: 0.0934109671920276
   }
 }
 start_time {
   seconds: 1703720771
   nanos: 924866275
 }
 end_time {
   seconds: 1703721086
 },
 id: "2"
 state: SUCCEEDED
 parameters {
   parameter_id: "alpha"
   value {
     number_value: 0.5713189670610704
   }
 }
 parameters {
   parameter_id: "eta"
   value {
     number_value: 0.29245142675654273
   }
 }
 parameters {
   parameter_id: "max_depth"
   value {
     number_value: 10.0
   }
 }
 final_measurement {
   step_count: 1
   metrics {
     metric_id: "Mean_square_log_error"
     value: 0.09319917807858441
   }
 }
 start_time {
   seconds: 1703720771
   nanos: 925036616
 }
 

In [119]:
best

id: "10"
state: SUCCEEDED
parameters {
  parameter_id: "alpha"
  value {
    number_value: 0.054789458485290723
  }
}
parameters {
  parameter_id: "eta"
  value {
    number_value: 0.6877740759969978
  }
}
parameters {
  parameter_id: "max_depth"
  value {
    number_value: 10.0
  }
}
final_measurement {
  step_count: 1
  metrics {
    metric_id: "Mean_square_log_error"
    value: 0.09261142805227456
  }
}
start_time {
  seconds: 1703722821
  nanos: 577550579
}
end_time {
  seconds: 1703723142
}

In [130]:
import pickle
from tempfile import TemporaryFile



In [138]:
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET)
blob = bucket.blob('01/01/models/20231227110602/10/model/xgb_model.pkl')


In [135]:
bucket

<Bucket: my-project-media-campaign-cost>

In [139]:
blob

<Blob: my-project-media-campaign-cost, 01/01/models/20231227110602/10/model/xgb_model.pkl, None>

In [140]:
with TemporaryFile() as temp_file:
    #download blob into temp file
    blob.download_to_file(temp_file)
    temp_file.seek(0)
    model=pickle.load(temp_file)

In [146]:
model.feature_importances_

array([0.02893353, 0.        , 0.03468262, 0.        , 0.0862874 ,
       0.09447674, 0.51019937, 0.09865638, 0.03014099, 0.05288786,
       0.03224003, 0.03149509], dtype=float32)